In [1]:
def alles_samen (hoogte, breedte, aantal_balken, aantal_keer):
    
    eindlijst_1 = []
    for x in range(0,aantal_keer):

        balken_database = database_gen (10, 4000, aantal_balken, 10)
        #print(balken_database)
        print('\n')

        balken_database_nieuw = [] 

        for balk in balken_database:
            if balk > hoogte:
                balken_database_nieuw.append(balk)

        balken_database_nieuw.sort()

        mega_lijst = diagonalen(hoogte, breedte, balken_database)
        print(len(mega_lijst))

        #optie met vaste HOH 
        vast_hoh (hoogte,breedte,600,balken_database,eindlijst_1)


        #welke varierende hoh afstand binnen 500 - 600 mm creert de minste afval (voorbeeld hoh 503, 598, 532)
        #stramien_niet_vast_min (hoogte,breedte,mega_lijst,balken_database,eindlijst_1)

        #hoh wordt elke keer random gegenereerd t/m gewenste breedte wordt behaald. na 10000x wordt de beste optie gekozen 
        #hsb_random_hoh (hoogte,breedte,mega_lijst,balken_database,eindlijst_1)

        #elke mogelijk optie (5mm range) wordt geprobeerd, kleiner dan 5 mm wordt de rekentijd te lang.
        #bruteforce_elke_optie (hoogte,breedte,mega_lijst,balken_database,eindlijst_1)
        print(eindlijst_1)
        
    print(eindlijst_1)

In [2]:
def database_gen (min_lengte, max_lengte, aantal_balken, stap_grootte):

    import random 
    import math

    balken_database = []
    for i in range(0,aantal_balken):
        n = random.randrange(min_lengte,max_lengte,stap_grootte)
        balken_database.append(n)

    balken_database.sort()
    return balken_database


In [3]:
def stramien_niet_vast_min (hoogte,breedte,mega_lijst,balken_database,eindlijst_1): 
    
    #deze pakt elke keer de hoh breedte met minste afval en gaat daar op door
    print('\n')
    print('dit is de functie 500-600 elke keer andere hoh met minste afval ')
    
    positie = 0
    while_counter = 0 
    marge = 0
    positie_huidige_balk = 0
    totaal_lengte_list = []
    balken_gebruikt = [] 
    balken_afval = [] 
    counter_eind = 0 
    beste_optie_afval = [] 
    beste_optie_lengte = []

    for data_set in mega_lijst:
    
        balken_database.remove(data_set[0])
        balken_database.remove(data_set[1])
        balken_gebruikt.append(data_set[0])
        balken_gebruikt.append(data_set[1])
        balken_afval.append(data_set[4])

        counter = 0 
        links = 0 
        start_breedte = 600
        totaal_breedte = 0 
        noodstop = 0 
        marge_hoogte = 0
        breedte_a = data_set[2]
        breedte_b = data_set[3]
        balken_sort = []
        balken_sort2 = []

        #zoekt de uiterste balken(links rechts)
        balken_database.sort()
        for x in range(0,4):
            for balk in balken_database[:]:
                if balk >= hoogte:
                    balken_gebruikt.append(balk)
                    afval = balk - hoogte
                    balken_afval.append(afval)
                    balken_database.remove(balk)
                    break
                    
        #zoekt de uiterste balken(boven & onder)
        boven_onder_balk (breedte_a,balken_database,balken_gebruikt,balken_afval) 
        boven_onder_balk (breedte_b,balken_database,balken_gebruikt,balken_afval)
        
        #zoekt het binnen frame hoh afstanden
        balk_binnen_minHOH(breedte_a,hoogte,balken_database,balken_gebruikt,balken_afval)
        balk_binnen_minHOH(breedte_b,hoogte,balken_database,balken_gebruikt,balken_afval)  

        print(counter_eind)    
        print(f'totaal balken gebruikt: {len(balken_gebruikt)}')
        print(f'totaal lengte balken gebruikt: {(sum(balken_gebruikt)/1000)} meter')
        print(f'lijst met afval stukken: {balken_afval}')
        print(f'totaal lengte afval: {(sum(balken_afval)/1000)} meter')
        
        beste_optie_afval.append(sum(balken_afval))
        beste_optie_lengte.append(sum(balken_gebruikt))
        
        for x in balken_gebruikt:
            balken_database.append(x)
            
        balken_gebruikt = []
        balken_afval = [] 
        balken_database.sort()
        counter_eind += 1 
        print(len(balken_database))
        print('\n')
        
    minst_afval = min(beste_optie_afval)
    minst_lengte = min(beste_optie_lengte)
    
    index_afval = beste_optie_afval.index(minst_afval)
    index_balken = beste_optie_lengte.index(minst_lengte)
    lengte_erbij = beste_optie_lengte[index_afval]
    
    eindlijst_1.append('min hoh')
    eindlijst_1.append(minst_afval)
    eindlijst_1.append(lengte_erbij)
    

    print('\n')
    print (f'dit is de optie met minste afval {index_afval}')
    print (f'dit is de optie met totaal lengte {index_balken}')
    
    


In [4]:
def balk_binnen_minHOH (breedte_totaal,hoogte,balken_database,balken_gebruikt,balken_afval):
    
    counter = 0 
    links = 0 
    start_breedte = 600
    totaal_breedte = 0 
    noodstop = 0 
    marge_hoogte = 0
    balken_sort = []
    balken_sort2 = []

    while totaal_breedte < (breedte_totaal - 600):
            
        for links in range(0,101): 
            nodig_hoogte = round(hoogte / (breedte_totaal / (start_breedte + totaal_breedte - links)))
            nodig_hoogte2 = hoogte - nodig_hoogte + marge_hoogte

            for balk in balken_database:
                if balk >= nodig_hoogte:
                    balk_a = balk
                    balken_database.remove(balk_a)
                    break

            for balk in balken_database:
                if balk >= nodig_hoogte2:
                    balk_b = balk
                    balken_database.append(balk_a)
                    break

            balk_afval = balk_a + balk_b - hoogte
            balk_koppel = [balk_a, balk_b, balk_afval,links]

            balken_sort.append(balk_koppel)
            balken_sort2.append(balk_afval)
            
            balken_database.sort()

        minste_afval = min(balken_sort2)
        index = balken_sort2.index(minste_afval)

        verder_gaan = balken_sort[index]

        balken_database.remove(verder_gaan[0])
        balken_database.remove(verder_gaan[1])
        balken_gebruikt.append(verder_gaan[0])
        balken_gebruikt.append(verder_gaan[1])
        balken_afval.append(verder_gaan[2])

        totaal_breedte = totaal_breedte + start_breedte - verder_gaan[3]

        balken_sort = [] 
        balken_sort2 = [] 

In [5]:
def diagonalen(hoogte, breedte, balken_database):
    start_breedte = round(breedte*0.25)
    eind_breedte = round(breedte*0.5) 
    counter_totaal = 0 
    counter_een = 0 
    counter_onder = 0 

    diagonalen_tijdelijk = [] 
    diagonalen_opties = [] 
    
    for huidige_breedte in range(start_breedte,eind_breedte+1):
        breedte_a = huidige_breedte
        breedte_b = breedte - huidige_breedte
        diagonaal_a = round(int((breedte_a**2 + hoogte**2)**0.5))
        diagonaal_b = round(int((breedte_b**2 + hoogte**2)**0.5))

        
        for balk in balken_database: 
            if balk >= diagonaal_a:
                afval_a = balk-diagonaal_a
                balk_a = balk
                balken_database.remove(balk)
                diagonalen_tijdelijk.append(balk)
                break

        for balk in balken_database: 
            if balk >= diagonaal_b:
                afval_b = balk-diagonaal_b
                balk_b = balk
                diagonalen_tijdelijk.append(balk)
                break

        if len(diagonalen_tijdelijk) != 2:
                balken_database.append(balk_a)
                counter_een += 1 

        #als er 2 balken zijn gevonden dan komen ze in een nieuwe database, anders volgende stap 
        if len(diagonalen_tijdelijk) == 2:
            totaal_afval = afval_a + afval_b
            balken_database.append(balk_a)
            diagonalen_tijdelijk.append(breedte_a)
            diagonalen_tijdelijk.append(breedte_b)
            diagonalen_tijdelijk.append(totaal_afval)
            code = str(balk_a)+'a'+str(balk_b)+'a'+str(totaal_afval)
            diagonalen_tijdelijk.append(code)
            counter_totaal += 1 
            
            if totaal_afval < 30:
                diagonalen_opties.append(diagonalen_tijdelijk)
                counter_onder += 1 
                
            totaal_afval = 0 

        diagonalen_tijdelijk = []
    
    print(f'dit is de counter: {counter_totaal}')
    print(f'counter lijst lengte 1 : {counter_een}')
    print(f'counter afval <50  : {counter_onder}')
    
    code_lijst = []
    diagonalen_opties2 = []
    
    for x in diagonalen_opties: 
        if x[5] not in code_lijst:
            diagonalen_opties2.append(x)
            code_lijst.append(x[5])
            
    for x in diagonalen_opties2:
        x.remove(x[5])
    
    return diagonalen_opties2

In [6]:
def boven_onder_balk (breedte_segment,balken_database,balken_gebruikt,balken_afval):
    for x in range(0,2):
            for balk in balken_database:
                    if balk >= breedte_segment:
                        balken_gebruikt.append(balk)
                        afval = balk - breedte_segment
                        balken_afval.append(afval)
                        balken_database.remove(balk)
                        break 

In [7]:
#werk pakt elke keer een random hoh, herhaald heel vaak en pakt de beste optie
def hsb_random_hoh (hoogte,breedte,mega_lijst,balken_database,eindlijst_1): 
    
    print('\n')
    print('deze pakt elke keer een random hoh')
    
    balken_database.sort()
    positie = 0
    while_counter = 0 
    marge = 0
    positie_huidige_balk = 0
    totaal_lengte_list = []
    balken_gebruikt = [] 
    balken_afval = []
    counter_eind = 0 
    beste_optie_afval = [] 
    beste_optie_lengte = []
    
    
    for data_set in mega_lijst:
        
        balken_database.remove(data_set[0])
        balken_database.remove(data_set[1])
        balken_gebruikt.append(data_set[0])
        balken_gebruikt.append(data_set[1])
        balken_afval.append(data_set[4])

        breedte_a = data_set[2]
        breedte_b = data_set[3]
    
        random_counter = 10000

        #zoekt de uiterste balken
        balken_database.sort()
        for x in range(0,4):
            for balk in balken_database[:]:
                if balk >= hoogte:
                    balken_gebruikt.append(balk)
                    afval = balk - hoogte
                    balken_afval.append(afval)
                    balken_database.remove(balk)
                    break

        boven_onder_balk (breedte_a,balken_database,balken_gebruikt,balken_afval)
        boven_onder_balk (breedte_b,balken_database,balken_gebruikt,balken_afval)
        
        balk_binnen_random_hoh (breedte_a,hoogte,balken_database,balken_gebruikt,balken_afval,random_counter)
        balk_binnen_random_hoh (breedte_b,hoogte,balken_database,balken_gebruikt,balken_afval,random_counter)
            
        print(counter_eind) 
        print(f'totaal balken gebruikt: {len(balken_gebruikt)}')
        print(f'totaal lengte balken gebruikt: {(sum(balken_gebruikt)/1000)} meter')
        print(f'lijst met afval stukken: {balken_afval}')
        print(f'totaal lengte balken afval: {(sum(balken_afval)/1000)} meter')
        
        beste_optie_afval.append(sum(balken_afval))
        beste_optie_lengte.append(sum(balken_gebruikt))
        
        for x in balken_gebruikt:
            balken_database.append(x)
            
        balken_gebruikt = []
        balken_afval = [] 
        balken_database.sort()
        counter_eind += 1 
        print(len(balken_database))
        print('\n')

    minst_afval = min(beste_optie_afval)
    minst_lengte = min(beste_optie_lengte)
    
    index_afval = beste_optie_afval.index(minst_afval)
    index_balken = beste_optie_lengte.index(minst_lengte)
    
    lengte_erbij = beste_optie_lengte[index_afval]
    
    eindlijst_1.append('random hoh')
    eindlijst_1.append(minst_afval)
    eindlijst_1.append(lengte_erbij)

    print('\n')
    print (f'dit is de optie met minste afval {index_afval}')
    print (f'dit is de optie met totaal lengte {index_balken}')


In [8]:
def balk_binnen_random_hoh (breedte_totaal,hoogte,balken_database,balken_gebruikt,balken_afval,random_counter):
    
    import random
    
    counter = 0 
    totaal_breedte = 0 
    balken_hoh_afstanden = []
    balken_hoh_varianten = []
    balk_afval_temp = []
    totaal_afval = 0 
    beste_optie = []
    balken_database.sort()

    while counter < random_counter:
        
            while totaal_breedte < (breedte_totaal - 600):

                start_breedte = random.randrange(500,600,1)
                nodig_hoogte = round(hoogte / (breedte_totaal / (start_breedte + totaal_breedte)))
                nodig_hoogte2 = hoogte - nodig_hoogte

                for balk in balken_database:
                    if balk >= nodig_hoogte:
                        balk_a = balk
                        balken_database.remove(balk_a)
                        break

                for balk in balken_database:
                    if balk >= nodig_hoogte2:
                        balk_b = balk
                        balken_database.remove(balk_b)
                        break

                balk_afval = balk_a + balk_b - hoogte
                balk_koppel = [balk_a, balk_b, balk_afval]
                balken_hoh_afstanden.append(balk_koppel)
                totaal_breedte = totaal_breedte + start_breedte
            
            balken_hoh_varianten.append(balken_hoh_afstanden)
            
            for balk in balken_hoh_afstanden:
                balken_database.append(balk[0])
                balken_database.append(balk[1])
                totaal_afval = totaal_afval + balk[2]
            
            balk_afval_temp.append(totaal_afval)
            totaal_afval = 0
            balken_database.sort() 
            counter += 1
            balken_hoh_afstanden = []
            totaal_breedte = 0
            
    minste_afval = min(balk_afval_temp)
    index = balk_afval_temp.index(minste_afval)
    beste_optie = balken_hoh_varianten[index]
    
    for balk in beste_optie:
        balken_database.remove(balk[0])
        balken_database.remove(balk[1])
        balken_gebruikt.append(balk[0])
        balken_gebruikt.append(balk[1])
        balken_afval.append(balk[2])

In [9]:
def bruteforce_elke_optie (hoogte,breedte,mega_lijst,balken_database,eindlijst_1): 

    print('\n')
    print('deze pakt elke optie met 5 mm speling')

    balken_database.sort()
    positie = 0
    while_counter = 0 
    marge = 0
    positie_huidige_balk = 0
    totaal_lengte_list = []
    balken_gebruikt = [] 
    balken_afval = []
    counter_eind = 0 
    beste_optie_afval = [] 
    beste_optie_lengte = []

    for data_set in mega_lijst:
    
        balken_database.remove(data_set[0])
        balken_database.remove(data_set[1])
        balken_gebruikt.append(data_set[0])
        balken_gebruikt.append(data_set[1])
        balken_afval.append(data_set[4])

        breedte_a = data_set[2]
        breedte_b = data_set[3]
        brute_factor = 5 
        
        balken_database.sort()
        for x in range(0,4):
            for balk in balken_database[:]:
                if balk >= hoogte:
                    balken_gebruikt.append(balk)
                    afval = balk - hoogte
                    balken_afval.append(afval)
                    balken_database.remove(balk)
                    break

        boven_onder_balk (breedte_a,balken_database,balken_gebruikt,balken_afval)
        boven_onder_balk (breedte_b,balken_database,balken_gebruikt,balken_afval)
        
        binnen_hoh_brute (breedte_a,hoogte,balken_database,balken_gebruikt,balken_afval,brute_factor)
        binnen_hoh_brute (breedte_b,hoogte,balken_database,balken_gebruikt,balken_afval,brute_factor)
            
        print(counter_eind) 
        print(f'totaal balken gebruikt: {len(balken_gebruikt)}')
        print(f'totaal lengte balken gebruikt: {(sum(balken_gebruikt)/1000)} meter')
        print(f'lijst met afval stukken: {balken_afval}')
        print(f'totaal lengte balken afval: {(sum(balken_afval)/1000)} meter')
        
        beste_optie_afval.append(sum(balken_afval))
        beste_optie_lengte.append(sum(balken_gebruikt))
        
        for x in balken_gebruikt:
            balken_database.append(x)
            
        balken_gebruikt = []
        balken_afval = [] 
        balken_database.sort()
        counter_eind += 1 
        print(len(balken_database))
        print('\n')

    minst_afval = min(beste_optie_afval)
    minst_lengte = min(beste_optie_lengte)
    
    index_afval = beste_optie_afval.index(minst_afval)
    index_balken = beste_optie_lengte.index(minst_lengte)
    
    lengte_erbij = beste_optie_lengte[index_afval]
    
    eindlijst_1.append('bruut')
    eindlijst_1.append(minst_afval)
    eindlijst_1.append(lengte_erbij)

    print('\n')
    print (f'dit is de optie met minste afval {index_afval}')
    print (f'dit is de optie met totaal lengte {index_balken}')

In [10]:
def binnen_hoh_brute (breedte_totaal,hoogte,balken_database,balken_gebruikt,balken_afval,brute_factor):
    
    totaal_breedte = 0 
    counter_list = [500,500,500,500]
    balken_hoh_afstanden = []
    balken_hoh_varianten = []
    balk_afval_temp = []
    totaal_afval = 0 
    beste_optie = []
    balken_database.sort()
    
    while counter_list[3] < 600:

        for optie in counter_list: 

            nodig_hoogte = round(hoogte / (breedte_totaal / (optie + totaal_breedte)))
            nodig_hoogte2 = hoogte - nodig_hoogte 

            for balk in balken_database:
                if balk >= nodig_hoogte:
                    balk_a = balk
                    balken_database.remove(balk_a)
                    break

            for balk in balken_database:
                if balk >= nodig_hoogte2:
                    balk_b = balk 
                    balken_database.remove(balk_b)
                    break

            balk_afval = balk_a + balk_b - hoogte
            balk_koppel = [balk_a, balk_b, balk_afval]
            balken_hoh_afstanden.append(balk_koppel)
            
            totaal_breedte = totaal_breedte + optie

            if totaal_breedte > (breedte_totaal - 600):
                totaal_breedte = 0 
                break
        
        totaal_breedte = 0 
        list_counter (counter_list,brute_factor)
        
        balken_hoh_varianten.append(balken_hoh_afstanden)

        for balk in balken_hoh_afstanden:
            balken_database.append(balk[0])
            balken_database.append(balk[1])
            totaal_afval = totaal_afval + balk[2]
        
        balk_afval_temp.append(totaal_afval)
        totaal_afval = 0
        balken_database.sort() 
        balken_hoh_afstanden = []
        totaal_breedte = 0
            
    minste_afval = min(balk_afval_temp)
    index = balk_afval_temp.index(minste_afval)
    beste_optie = balken_hoh_varianten[index]
    
    for balk in beste_optie:
        balken_database.remove(balk[0])
        balken_database.remove(balk[1])
        balken_gebruikt.append(balk[0])
        balken_gebruikt.append(balk[1])
        balken_afval.append(balk[2])

In [11]:
def list_counter (test_list,brute_factor):
    if test_list[0]  == 600:
        test_list[1] = test_list[1] + brute_factor 
        test_list[0] = 500 - brute_factor

    if test_list[1]  == 600 + brute_factor:
        test_list[2] = test_list[2] + brute_factor 
        test_list[1] = 500
        test_list[0] = 500 - brute_factor

    if test_list[2]  == 600 + brute_factor:
        test_list[3] = test_list[3] + brute_factor
        test_list[2] = 500
        test_list[1] = 500
        test_list[0] = 500 - brute_factor

    test_list[0] = test_list[0]+brute_factor

In [12]:
def vast_hoh (hoogte,breedte_wand,stramien,balken_database,eindlijst_1):

    import random 
    import math

    
    balken_database.sort() 
    beste_optie_afval = []
    beste_optie_lengte = []
    counter = 0

 
    print('\n')
    print(f' dit is hoh {stramien}')

    segmenten = math.floor(breedte_wand / stramien)
    breedte = segmenten * stramien

    counter_up = 1 
    counter_down = segmenten-1

    aantal_keer = math.floor(segmenten / 2)
    lengte_a = 0 
    lengte_b = 0

    totaal_lengte_list_tijdelijk = []
    totaal_lengte_list = []
    balken_gebruikt = []
    balken_afval = [] 

    for i in range(0,aantal_keer):
        schuine_lengte_a = round((hoogte**2 + (stramien*counter_up)**2)**0.5)
        schuine_lengte_b = round((hoogte**2 + (stramien*counter_down)**2)**0.5)

        #zoektocht balk linkerdeel segmenten
        for x in balken_database: 
            if x >= schuine_lengte_a:
                lengte_a = x-schuine_lengte_a
                balk_a = x
                balken_database.remove(x)
                totaal_lengte_list_tijdelijk.append(x)
                break

        #zoektocht balk rechterdeel segmenten
        for x in balken_database: 
            if x >= schuine_lengte_b:
                lengte_b = x-schuine_lengte_b
                totaal_lengte_list_tijdelijk.append(x)
                break

        if len(totaal_lengte_list_tijdelijk) == 1:
            balken_database.append(balk_a)

        #als er 2 balken zijn gevonden dan komen ze in een nieuwe database, anders volgende stap 
        if len(totaal_lengte_list_tijdelijk) == 2:
            totaal_afval = lengte_a + lengte_b
            balken_database.append(balk_a)
            totaal_lengte_list_tijdelijk.append(counter_up)
            totaal_lengte_list_tijdelijk.append(counter_down)
            totaal_lengte_list_tijdelijk.append(totaal_afval)
            totaal_lengte_list.append(totaal_lengte_list_tijdelijk)

        #reset tijdelijke list
        totaal_lengte_list_tijdelijk = []
        counter_up += 1 
        counter_down -= 1 

    print(totaal_lengte_list)

    for data_set in totaal_lengte_list: 

        balken_gebruikt.append(data_set[0])
        balken_gebruikt.append(data_set[1])
        balken_afval.append(data_set[4])
        balken_database.remove(data_set[0])
        balken_database.remove(data_set[1])
        gekozen_segmenten = [data_set[2],data_set[3]]
        balken_database.sort()
        teller = 0
        aantal_buitenste_balk = len(gekozen_segmenten)*2 

        #zoekt de uiterste balken
        for i in balken_database[:]:
            if i >= hoogte and teller < aantal_buitenste_balk:
                balken_gebruikt.append(i)
                afval = i - hoogte
                balken_afval.append(afval)
                balken_database.remove(i)
                teller +=1

        for y in range(0,len(gekozen_segmenten)):
            for x in range(0,len(gekozen_segmenten)):
                benodigte_totaal_breedte = gekozen_segmenten[y] * stramien
                for balk in balken_database[:]:
                        if balk >= benodigte_totaal_breedte:
                            balken_gebruikt.append(balk)
                            afval = balk - benodigte_totaal_breedte
                            balken_afval.append(afval)
                            balken_database.remove(balk)
                            break 

        for y in range(0,len(gekozen_segmenten)):
            for x in range(1,gekozen_segmenten[y]):

                split_hoogte = round(hoogte / ((gekozen_segmenten[y] * stramien) / (x * stramien)))
                restant_split_hoogte = hoogte - split_hoogte

                for balk in balken_database:
                    if balk >= split_hoogte:
                        balken_gebruikt.append(balk)
                        afval = balk - split_hoogte
                        balken_afval.append(afval)
                        balken_database.remove(balk)
                        break

                for balk in balken_database:
                    if balk >= restant_split_hoogte:
                        balken_gebruikt.append(balk)
                        afval = balk - restant_split_hoogte
                        balken_afval.append(afval)
                        balken_database.remove(balk)
                        break 

        print('\n')
        print(counter)
        print(gekozen_segmenten)
        print(f'aantal balken gebruikt {len(balken_gebruikt)}')
        print(f'totaal lengte balken gebruikt {(sum(balken_gebruikt)/1000)} meter')
        print(f'totaal afval: {(sum(balken_afval)/1000)} meter')
        print(f'percentage afval: {round((sum(balken_afval)/1000)/(sum(balken_gebruikt)/1000)*100,2)} %')
        print(balken_afval)
        print(f'breedte wand: {breedte}')

        beste_optie_afval.append(sum(balken_afval))
        beste_optie_lengte.append(sum(balken_gebruikt))

        for x in balken_gebruikt: 
            balken_database.append(x) 

        balken_database.sort()
        balken_afval = []
        balken_gebruikt = [] 
        counter += 1 

    minst_afval = min(beste_optie_afval)
    minst_lengte = min(beste_optie_lengte)
    
    index_afval = beste_optie_afval.index(minst_afval)
    index_balken = beste_optie_lengte.index(minst_lengte)
    
    lengte_erbij = beste_optie_lengte[index_afval]
    
    eindlijst_1.append('vast hoh')
    eindlijst_1.append(minst_afval)
    eindlijst_1.append(lengte_erbij)
    
    print('\n')
    print (f'dit is de optie met minste afval {index_afval}')
    print (f'dit is de optie met totaal lengte {index_balken}')

In [18]:
#dit pakt alle opties samen om te vergelijken welke het beste is. 
alles_samen(2500,4800,1000,1)



dit is de counter: 711
counter lijst lengte 1 : 490
counter afval <50  : 34
16


 dit is hoh 600
[[3090, 3910, 3, 5, 14], [3470, 3470, 4, 4, 8]]


0
[3, 5]
aantal balken gebruikt 22
totaal lengte balken gebruikt 41.67 meter
totaal afval: 0.084 meter
percentage afval: 0.2 %
[14, 0, 0, 10, 30, 0, 0, 0, 10, 7, 3, 3, 7, 0, 0, 0, 0, 0, 0, 0, 0]
breedte wand: 4800


1
[4, 4]
aantal balken gebruikt 22
totaal lengte balken gebruikt 41.65 meter
totaal afval: 0.118 meter
percentage afval: 0.28 %
[8, 0, 0, 10, 30, 0, 0, 0, 0, 5, 5, 0, 0, 5, 5, 5, 15, 0, 0, 15, 15]
breedte wand: 4800


dit is de optie met minste afval 0
dit is de optie met totaal lengte 1
['vast hoh', 84, 41670]
['vast hoh', 84, 41670]


In [14]:
['vast hoh', 134, 41720, 'min hoh', 81, 44130, 'random hoh', 86, 46620, 'bruut', 81, 44130]

['vast hoh',
 134,
 41720,
 'min hoh',
 81,
 44130,
 'random hoh',
 86,
 46620,
 'bruut',
 81,
 44130]

In [15]:
['vast hoh', 84, 41670, 'min hoh', 25, 44060, 'random hoh', 35, 44070, 'vast hoh', 54, 41640, 'min hoh', 30, 44060, 'random hoh', 33, 44090, 'vast hoh', 134, 41720, 'min hoh', 70, 44120, 'random hoh', 80, 44130, 'vast hoh', 84, 41670, 'min hoh', 72, 44160, 'random hoh', 82, 44170, 'vast hoh', 104, 41690, 'min hoh', 21, 44080, 'random hoh', 31, 46590, 'vast hoh', 94, 41680, 'min hoh', 54, 44100, 'random hoh', 54, 44100, 'vast hoh', 94, 41680, 'min hoh', 32, 46620, 'random hoh', 52, 46640, 'vast hoh', 114, 41700, 'min hoh', 25, 44060, 'random hoh', 35, 44070, 'vast hoh', 114, 41700, 'min hoh', 24, 46610, 'random hoh', 34, 46620, 'vast hoh', 84, 41670, 'min hoh', 73, 44120, 'random hoh', 73, 44120]

['vast hoh',
 84,
 41670,
 'min hoh',
 25,
 44060,
 'random hoh',
 35,
 44070,
 'vast hoh',
 54,
 41640,
 'min hoh',
 30,
 44060,
 'random hoh',
 33,
 44090,
 'vast hoh',
 134,
 41720,
 'min hoh',
 70,
 44120,
 'random hoh',
 80,
 44130,
 'vast hoh',
 84,
 41670,
 'min hoh',
 72,
 44160,
 'random hoh',
 82,
 44170,
 'vast hoh',
 104,
 41690,
 'min hoh',
 21,
 44080,
 'random hoh',
 31,
 46590,
 'vast hoh',
 94,
 41680,
 'min hoh',
 54,
 44100,
 'random hoh',
 54,
 44100,
 'vast hoh',
 94,
 41680,
 'min hoh',
 32,
 46620,
 'random hoh',
 52,
 46640,
 'vast hoh',
 114,
 41700,
 'min hoh',
 25,
 44060,
 'random hoh',
 35,
 44070,
 'vast hoh',
 114,
 41700,
 'min hoh',
 24,
 46610,
 'random hoh',
 34,
 46620,
 'vast hoh',
 84,
 41670,
 'min hoh',
 73,
 44120,
 'random hoh',
 73,
 44120]

In [16]:
x = ['vast hoh', 84, 41670, 'min hoh', 25, 44060, 'random hoh', 35, 44070, 'vast hoh', 54, 41640, 'min hoh', 30, 44060, 'random hoh', 33, 44090, 'vast hoh', 134, 41720, 'min hoh', 70, 44120, 'random hoh', 80, 44130, 'vast hoh', 84, 41670, 'min hoh', 72, 44160, 'random hoh', 82, 44170, 'vast hoh', 104, 41690, 'min hoh', 21, 44080, 'random hoh', 31, 46590, 'vast hoh', 94, 41680, 'min hoh', 54, 44100, 'random hoh', 54, 44100, 'vast hoh', 94, 41680, 'min hoh', 32, 46620, 'random hoh', 52, 46640, 'vast hoh', 114, 41700, 'min hoh', 25, 44060, 'random hoh', 35, 44070, 'vast hoh', 114, 41700, 'min hoh', 24, 46610, 'random hoh', 34, 46620, 'vast hoh', 84, 41670, 'min hoh', 73, 44120, 'random hoh', 73, 44120]

x

In [17]:
x

['vast hoh',
 84,
 41670,
 'min hoh',
 25,
 44060,
 'random hoh',
 35,
 44070,
 'vast hoh',
 54,
 41640,
 'min hoh',
 30,
 44060,
 'random hoh',
 33,
 44090,
 'vast hoh',
 134,
 41720,
 'min hoh',
 70,
 44120,
 'random hoh',
 80,
 44130,
 'vast hoh',
 84,
 41670,
 'min hoh',
 72,
 44160,
 'random hoh',
 82,
 44170,
 'vast hoh',
 104,
 41690,
 'min hoh',
 21,
 44080,
 'random hoh',
 31,
 46590,
 'vast hoh',
 94,
 41680,
 'min hoh',
 54,
 44100,
 'random hoh',
 54,
 44100,
 'vast hoh',
 94,
 41680,
 'min hoh',
 32,
 46620,
 'random hoh',
 52,
 46640,
 'vast hoh',
 114,
 41700,
 'min hoh',
 25,
 44060,
 'random hoh',
 35,
 44070,
 'vast hoh',
 114,
 41700,
 'min hoh',
 24,
 46610,
 'random hoh',
 34,
 46620,
 'vast hoh',
 84,
 41670,
 'min hoh',
 73,
 44120,
 'random hoh',
 73,
 44120]